# Demo de Extracción desde Socrata

En este notebook se muestra paso a paso cómo utilizar la clase `SocrataDatasetLoader` para extraer datos de Socrata, aplicar validaciones de gobernanza y calidad, y registrar la metadata asociada al proceso. 

Cada celda está comentada para que puedas entender el propósito y el funcionamiento de cada paso.

## Librerias

In [48]:
# Importamos las librerías y módulos necesarios.
import logging
import pandas as pd
from sodapy import Socrata
import sys
import json

### Configuración del entorno de ejecución

In [21]:
from config.notebook_location import find_project_root

In [23]:
# Definir el nombre del directorio del proyecto
project_name = "personal-library"

# Encontrar la raíz del proyecto
project_root = find_project_root(project_name)

# Agregar la raíz del proyecto al principio de sys.path para facilitar las importaciones
sys.path.insert(0, project_root)
print("Project root added to sys.path:", project_root)

Project root added to sys.path: d:\Espacios de trabajo\personal-library


### Librerías personales

In [29]:
# Importamos las clases definidas en nuestro sistema de ingesta.
from ingestion.sources.from_socrata import SocrataDatasetLoader
from ingestion.governance.policy_loader import load_policy
from ingestion.governance.engine import GovernanceEngine
from ingestion.base.metadata_logger import MetadataLogger

## Proceso

In [30]:
# Configuramos el logging para ver información detallada durante la ejecución.
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

**1. Inicialización del Cliente Socrata**
 
Se crea una instancia del cliente de Socrata apuntando al dominio de datos. En este ejemplo, usamos "www.datos.gov.co". 
Si tu dataset requiere autenticación, provee las credenciales necesarias (aquí se usa `None` como placeholder).

In [31]:
# Inicializamos el cliente de Socrata.
client = Socrata("www.datos.gov.co", None)

2025-03-21 18:28:46,453 - root - WARNING - Requests made without an app_token will be subject to strict throttling limits.


**2. Configuración del Loader y Extracción de Datos**
 
Se crea una instancia de `SocrataDatasetLoader` y se utiliza para extraer datos de un dataset específico.
- **dataset_code:** Código del dataset en Socrata.
- **filters:** Filtros de consulta; en este ejemplo, extraemos registros donde `fecha_de_firma` es mayor o igual a "2023-01-01".
- **limit:** Número máximo de registros a extraer.

In [32]:
# Creamos la instancia del loader para Socrata.
loader = SocrataDatasetLoader(client)

In [33]:
# Definimos los parámetros para la extracción.
dataset_code = "jbjy-vk9h"  # Código del dataset en Socrata
filters = {"fecha_de_firma": (">=", "2023-01-01")}
limit = 100

In [34]:
# Realizamos la carga de datos.
df = loader.load_data(dataset_code=dataset_code, filters=filters, limit=limit)

2025-03-21 18:30:18,812 - ingestion.sources.from_socrata - DEBUG - Executing query: SELECT * WHERE fecha_de_firma >= '2023-01-01' LIMIT 100
2025-03-21 18:30:18,822 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): www.datos.gov.co:443
2025-03-21 18:30:21,612 - urllib3.connectionpool - DEBUG - https://www.datos.gov.co:443 "GET /resource/jbjy-vk9h.json?%24query=SELECT+%2A+WHERE+fecha_de_firma+%3E%3D+%272023-01-01%27+LIMIT+100 HTTP/1.1" 200 None
2025-03-21 18:30:21,728 - ingestion.sources.from_socrata - INFO - Loaded 100 rows from dataset jbjy-vk9h


In [35]:
# Mostramos las primeras filas del DataFrame extraído.
df.head()

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,nombre_ordenador_del_gasto,tipo_de_documento_ordenador_del_gasto,n_mero_de_documento_ordenador_del_gasto,nombre_supervisor,tipo_de_documento_supervisor,n_mero_de_documento_supervisor,nombre_ordenador_de_pago,tipo_de_documento_ordenador_de_pago,n_mero_de_documento_ordenador_de_pago,fecha_de_notificaci_n_de_prorrogaci_n
0,HOSPITAL SAN JUAN DE DIOS DE HONDA ESE,890700666,Tolima,Honda,"Colombia, Tolima , Honda",Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,CO1.BDOS.3681223,...,MANUEL ALFONSO GONZaLEZ CANTOR,Cédula de Ciudadanía,79393172,MARTHA LUCIA OSORIO RAMIREZ,Cédula de Ciudadanía,38281318,No definido,No definido,No definido,NaN
1,HOSPITAL DE CASTILLA LA NUEVA EMPRESA SOCIAL D...,900004059,Meta,Castilla La Nueva,"Colombia, Meta , Castilla La Nueva",Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,CO1.BDOS.3690528,...,ROSA MARIA JIMENEZ BAQUERO,Cédula de Ciudadanía,40370893,LUIS ENRIQUE BARON TELLO,Cédula de Ciudadanía,1121827353,No definido,No definido,No definido,NaN
2,HOSPITAL SAN JUAN DE DIOS DE HONDA ESE,890700666,Tolima,Honda,"Colombia, Tolima , Honda",Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,CO1.BDOS.3677624,...,MANUEL ALFONSO GONZaLEZ CANTOR,Cédula de Ciudadanía,79393172,ANGELA MARLEN SALGUERO RAMIREZ,Cédula de Ciudadanía,1110526365,No definido,No definido,No definido,NaN
3,EMPRESA SOCIAL DEL ESTADO HOSPITAL EL CARMEN 1,8909821018,Antioquia,Amalfi,"Colombia, Antioquia , Amalfi",Territorial,Salud y Protección Social,Ejecutivo,Descentralizada,CO1.BDOS.3688365,...,LICINIA DEL CARMEN RAVE BERMUDEZ,Cédula de Ciudadanía,42876516,ANGEL ERNESTO FRANCO HENAO,Cédula de Ciudadanía,70252196,No definido,No definido,No definido,NaN
4,MUNICIPIO DE ITAGUI,890980093,Antioquia,Itagui,"Colombia, Antioquia , Itagui",Territorial,"Vivienda, Ciudad y Territorio",Ejecutivo,Centralizada,CO1.BDOS.3671441,...,No definido,No definido,No definido,No definido,No definido,No definido,No definido,No definido,No definido,NaN


In [47]:
df.columns

Index(['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad',
       'localizaci_n', 'orden', 'sector', 'rama', 'entidad_centralizada',
       'proceso_de_compra', 'id_contrato', 'referencia_del_contrato',
       'estado_contrato', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'tipo_de_contrato',
       'modalidad_de_contratacion', 'justificacion_modalidad_de',
       'fecha_de_firma', 'fecha_de_inicio_del_contrato',
       'fecha_de_fin_del_contrato', 'condiciones_de_entrega',
       'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
       'es_grupo', 'es_pyme', 'habilita_pago_adelantado', 'liquidaci_n',
       'obligaci_n_ambiental', 'obligaciones_postconsumo', 'reversion',
       'origen_de_los_recursos', 'destino_gasto', 'valor_del_contrato',
       'valor_de_pago_adelantado', 'valor_facturado',
       'valor_pendiente_de_pago', 'valor_pagado', 'valor_amortizado',
       'valor_pendiente_de', 'valor_pendiente_de_ejecucion', 'estado_bpin',
 

**3. Carga de la Política de Gobernanza**

La política define las reglas de calidad y gobernabilidad que se aplicarán al dataset.

Se carga desde un archivo YAML. Asegúrate de que el archivo `s2_contracts.yaml` exista en la ruta indicada.

In [54]:
# Cargamos la política de gobernanza desde un archivo YAML.
policy_path = "../../ingestion/governance/policies/s2_contracts.yaml"
policy = load_policy(policy_path)

# Imprimimos la política cargada para verificar su contenido.
print("Política de Gobernanza Cargada:")
print(json.dumps(policy, indent=4))

Política de Gobernanza Cargada:
{
    "required_fields": [
        "id",
        "fecha",
        "total"
    ],
    "expected_types": {
        "fecha": "datetime64",
        "total": "float64"
    },
    "rules": {
        "total": {
            "rule": "range",
            "min": 0,
            "max": 1000000
        },
        "total_2": "unique",
        "descripcion": {
            "rule": "string_length",
            "min_length": 5,
            "max_length": 255
        },
        "email": {
            "rule": "regex",
            "pattern": "^[\\w\\.-]+@[\\w\\.-]+\\.\\w+$"
        },
        "campo_importante": "not_null",
        "fecha": "not_future"
    },
    "inter_field_rules": [
        {
            "rule": "date_order",
            "fields": [
                "fecha_inicio",
                "fecha_fin"
            ]
        }
    ]
}


**4. Validación del Dataset con el Motor de Gobernanza**

Se utiliza la clase `GovernanceEngine` para aplicar las validaciones definidas en la política al DataFrame.

Se generan reportes que incluyen errores y advertencias detectadas en la ingesta.

In [45]:
# Creamos el motor de gobernanza y validamos el DataFrame.
engine = GovernanceEngine(df, policy)
validated_df, report = engine.validate()

# Imprimimos el reporte de validación.
print("Reporte de Gobernanza y Validación:")
print(report)

KeyError: 'total_2'

**5. Registro de Metadata y Auditoría**
 
Utilizamos la clase `MetadataLogger` para registrar la metadata del proceso de ingesta, incluyendo:

- Información del loader (número de filas, estado, filtros aplicados, etc.)

- Reporte de gobernanza (errores y advertencias)

La metadata se guarda en un archivo Parquet para su posterior auditoría y seguimiento.

In [46]:
# Inicializamos el MetadataLogger.
metadata_logger = MetadataLogger(report_path="reports/demo_socrata.parquet")

# Registramos la metadata combinando la información del loader y el reporte de validación.
metadata_logger.log({**loader.metadata, **report})

# Guardamos el log en el archivo especificado.
metadata_logger.save()

print("Metadata registrada y audit log guardado.")

NameError: name 'report' is not defined

## Conclusión
 
En este demo se ha mostrado:

- Cómo inicializar el cliente de Socrata.

- Cómo configurar y utilizar el `SocrataDatasetLoader` para extraer datos.

- Cómo cargar y aplicar una política de gobernanza a través del `GovernanceEngine`.

- Cómo registrar la metadata del proceso de ingesta utilizando `MetadataLogger`.

Este flujo modular y bien documentado garantiza la reutilización, calidad y trazabilidad de la ingesta de datos desde Socrata.
